# 🔧 TypeScript 미사용 변수 자동 해결 솔루션

## 📋 프로젝트 개요
**PosMul Platform**의 TypeScript 코드에서 발생하는 `@typescript-eslint/no-unused-vars` 에러를 체계적으로 분석하고 자동으로 해결하는 실습 노트북입니다.

### 🎯 목표
- ESLint 미사용 변수 에러 분석 및 이해
- 자동화된 해결 방안 구현
- CI/CD 파이프라인 통합
- VSCode 워크플로우 최적화

### 📁 대상 파일들
- `c:\G\posmul\src\bounded-contexts\prediction\domain\services\incentive-alignment.service.ts`
- `c:\G\posmul\src\bounded-contexts\prediction\domain\services\pmp-pmc-conversion.engine.ts`
- `c:\G\posmul\src\bounded-contexts\prediction\domain\services\portfolio-optimization.service.ts`
- `c:\G\posmul\src\shared\types\economic-system.ts`

---

In [1]:
# 필요한 라이브러리 import
import os
import re
import json
import subprocess
from pathlib import Path
from typing import List, Dict, Tuple
import pandas as pd

# 프로젝트 루트 경로 설정
PROJECT_ROOT = Path(r"c:\G\posmul")
print(f"프로젝트 루트: {PROJECT_ROOT}")
print(f"현재 작업 디렉토리: {os.getcwd()}")

# 프로젝트 구조 확인
if PROJECT_ROOT.exists():
    print("✅ 프로젝트 경로 확인됨")
else:
    print("❌ 프로젝트 경로를 찾을 수 없습니다")

프로젝트 루트: c:\G\posmul
현재 작업 디렉토리: c:\G\posmul\notebooks
✅ 프로젝트 경로 확인됨


## 3. 미사용 변수 에러 원인 분석

### 3.1 '@typescript-eslint/no-unused-vars' 에러란?
- **정의**: 선언되었지만 실제로 사용되지 않는 변수, 매개변수, import 등을 탐지하는 ESLint 규칙
- **목적**: 코드 품질 향상, 번들 크기 최적화, 개발자 실수 방지
- **TypeScript 환경**: TypeScript 컴파일러와 ESLint가 함께 작동하여 더 정확한 분석 제공

### 3.2 현재 프로젝트에서 발생한 에러 유형

#### 🔍 발견된 에러 패턴:
1. **미사용 매개변수**: `_config`, `_relationship`, `_params` 등
2. **미사용 로컬 변수**: `solutionConfig`, `userProfile` 등  
3. **미사용 타입 정의**: `AgencyCostScore` 등
4. **언더스코어 접두사 오용**: ESLint 설정에 따라 `_` 접두사도 에러로 간주될 수 있음

#### 📊 에러 발생 파일 현황:
- `incentive-alignment.service.ts`: 8개 에러
- `pmp-pmc-conversion.engine.ts`: 8개 에러  
- `portfolio-optimization.service.ts`: 1개 에러
- `economic-system.ts`: 1개 에러

In [3]:
# 실제 프로젝트에서 발생한 미사용 변수 에러 예시를 분석해보겠습니다.

# 프로젝트 경로 재정의 (이전 셀 실행이 필요할 수 있음)
from pathlib import Path
import re

project_path = Path(r"c:\G\posmul")

# 1. incentive-alignment.service.ts 파일의 에러 분석
incentive_alignment_file = project_path / "src" / "bounded-contexts" / "prediction" / "domain" / "services" / "incentive-alignment.service.ts"

if incentive_alignment_file.exists():
    with open(incentive_alignment_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print("📄 incentive-alignment.service.ts 파일 분석")
    print("=" * 60)
    print(f"파일 크기: {len(content)} 문자")
    print(f"라인 수: {len(content.splitlines())} 줄")
    
    # 미사용 매개변수 패턴 찾기
    
    # 미사용 매개변수 패턴 (_로 시작하는 매개변수)
    unused_params = re.findall(r'_\w+:', content)
    print(f"\n🔍 언더스코어 매개변수: {len(unused_params)}개")
    for param in set(unused_params):
        print(f"  - {param}")
    
    # console.log 패턴 찾기 (임시 디버깅 코드)
    console_logs = re.findall(r'console\.log\([^)]+\)', content)
    print(f"\n📝 console.log 사용: {len(console_logs)}개")
    for log in console_logs[:3]:  # 최대 3개만 표시
        print(f"  - {log[:50]}...")
else:
    print("❌ incentive-alignment.service.ts 파일을 찾을 수 없습니다.")

📄 incentive-alignment.service.ts 파일 분석
파일 크기: 27960 문자
라인 수: 915 줄

🔍 언더스코어 매개변수: 9개
  - _solutionConfig:
  - _config:
  - _reduction:
  - _userId:
  - _relationship:

📝 console.log 사용: 6개
  - console.log('Generating prevention actions for ris...
  - console.log('Developing selection strategy for', c...
  - console.log('Calculating effectiveness for', incen...


## 4. ESLint 설정 확인 및 수정 실습

### 4.1 현재 ESLint 설정 분석
프로젝트의 `eslint.config.mjs` 파일을 확인하여 `@typescript-eslint/no-unused-vars` 규칙이 어떻게 설정되어 있는지 분석합니다.

### 4.2 미사용 변수 규칙 최적화 방법
1. **argsIgnorePattern**: `^_`로 시작하는 매개변수 무시
2. **varsIgnorePattern**: 특정 패턴의 변수 무시  
3. **ignoreRestSiblings**: Rest 연산자 사용 시 형제 속성 무시
4. **destructuredArrayIgnorePattern**: 배열 구조분해에서 특정 요소 무시

### 4.3 권장 ESLint 설정 예시
```javascript
"@typescript-eslint/no-unused-vars": [
  "error",
  {
    "argsIgnorePattern": "^_",
    "varsIgnorePattern": "^_",
    "caughtErrorsIgnorePattern": "^_",
    "destructuredArrayIgnorePattern": "^_",
    "ignoreRestSiblings": true
  }
]
```

In [4]:
# ESLint 설정 파일 분석 및 수정

# 1. 현재 ESLint 설정 파일 확인
eslint_config_file = project_path / "eslint.config.mjs"

print("🔧 ESLint 설정 파일 분석")
print("=" * 50)

if eslint_config_file.exists():
    with open(eslint_config_file, 'r', encoding='utf-8') as f:
        eslint_content = f.read()
    
    print(f"✅ ESLint 설정 파일 발견: {eslint_config_file}")
    print(f"파일 크기: {len(eslint_content)} 문자")
    
    # no-unused-vars 규칙 찾기
    if "no-unused-vars" in eslint_content:
        print("\n🔍 no-unused-vars 규칙이 설정되어 있습니다.")
        
        # 해당 부분 추출
        lines = eslint_content.splitlines()
        for i, line in enumerate(lines):
            if "no-unused-vars" in line:
                print(f"라인 {i+1}: {line.strip()}")
                # 주변 라인도 표시 (설정 구조 파악)
                start = max(0, i-2)
                end = min(len(lines), i+5)
                print("\n📋 설정 컨텍스트:")
                for j in range(start, end):
                    marker = "👉 " if j == i else "   "
                    print(f"{marker}{j+1:3}: {lines[j]}")
                break
    else:
        print("\n⚠️  no-unused-vars 규칙이 명시적으로 설정되지 않았습니다.")
        print("기본 설정이 사용되고 있을 수 있습니다.")
else:
    print("❌ ESLint 설정 파일을 찾을 수 없습니다.")

# 2. package.json에서 ESLint 관련 의존성 확인
package_json = project_path / "package.json"
if package_json.exists():
    with open(package_json, 'r', encoding='utf-8') as f:
        import json
        package_data = json.load(f)
    
    print(f"\n📦 ESLint 관련 의존성:")
    
    # devDependencies에서 ESLint 관련 패키지 찾기
    dev_deps = package_data.get('devDependencies', {})
    eslint_deps = {k: v for k, v in dev_deps.items() if 'eslint' in k.lower()}
    
    for pkg, version in eslint_deps.items():
        print(f"  - {pkg}: {version}")
    
    # scripts에서 ESLint 명령어 확인
    scripts = package_data.get('scripts', {})
    eslint_scripts = {k: v for k, v in scripts.items() if 'eslint' in v.lower() or 'lint' in k.lower()}
    
    if eslint_scripts:
        print(f"\n🏃 ESLint 관련 스크립트:")
        for script, command in eslint_scripts.items():
            print(f"  - {script}: {command}")
else:
    print("\n❌ package.json 파일을 찾을 수 없습니다.")

🔧 ESLint 설정 파일 분석
✅ ESLint 설정 파일 발견: c:\G\posmul\eslint.config.mjs
파일 크기: 393 문자

⚠️  no-unused-vars 규칙이 명시적으로 설정되지 않았습니다.
기본 설정이 사용되고 있을 수 있습니다.

📦 ESLint 관련 의존성:
  - @eslint/eslintrc: ^3
  - eslint: ^9
  - eslint-config-next: 15.3.4

🏃 ESLint 관련 스크립트:
  - lint: next lint


## 5. 자동 탐지 및 제거 도구 실습

### 5.1 도구별 특징 비교

| 도구 | 장점 | 단점 | 사용 시나리오 |
|------|------|------|---------------|
| **ESLint --fix** | 설정된 규칙 자동 수정, IDE 통합 | 복잡한 케이스 처리 한계 | 일반적인 미사용 변수 |
| **ts-prune** | TypeScript 전용, 정확한 분석 | 수동 삭제 필요 | export된 미사용 코드 탐지 |
| **ts-unused-exports** | 빠른 분석, CI/CD 통합 용이 | 런타임 참조 놓칠 수 있음 | 대규모 프로젝트 |
| **VSCode Quick Fix** | 실시간, 사용자 친화적 | 파일별 개별 처리 | 개발 중 실시간 수정 |

### 5.2 각 도구의 실제 사용법
다음 셀에서 각 도구를 실제로 사용해보며 장단점을 비교분석합니다.

In [5]:
# 자동 탐지 및 제거 도구 실습

import subprocess
import json
import os

print("🛠️  자동 미사용 변수 탐지 및 제거 도구 실습")
print("=" * 60)

# 1. ESLint로 미사용 변수 에러 확인
print("\n1️⃣ ESLint로 미사용 변수 에러 확인")
print("-" * 40)

try:
    # 프로젝트 디렉토리로 이동하여 ESLint 실행
    os.chdir(str(project_path))
    
    # ESLint 실행 (JSON 포맷으로 결과 받기)
    result = subprocess.run(
        ['npx', 'eslint', 'src/', '--format', 'json'],
        capture_output=True,
        text=True,
        shell=True
    )
    
    if result.returncode != 0:
        try:
            eslint_data = json.loads(result.stdout)
            
            # 미사용 변수 에러만 필터링
            unused_var_errors = []
            for file_result in eslint_data:
                file_path = file_result['filePath']
                for message in file_result['messages']:
                    if 'no-unused-vars' in message.get('ruleId', ''):
                        unused_var_errors.append({
                            'file': file_path.replace(str(project_path), ''),
                            'line': message['line'],
                            'message': message['message'],
                            'severity': message['severity']
                        })
            
            print(f"🔍 총 {len(unused_var_errors)}개의 미사용 변수 에러 발견")
            
            # 파일별 에러 개수 집계
            file_error_count = {}
            for error in unused_var_errors:
                file_name = error['file']
                file_error_count[file_name] = file_error_count.get(file_name, 0) + 1
            
            print("\n📊 파일별 에러 분포:")
            for file_name, count in sorted(file_error_count.items(), key=lambda x: x[1], reverse=True)[:10]:
                print(f"  - {file_name}: {count}개")
                
        except json.JSONDecodeError:
            print("⚠️  ESLint 결과를 JSON으로 파싱할 수 없습니다.")
            print(f"ESLint 출력: {result.stdout[:200]}...")
    else:
        print("✅ ESLint 에러가 없습니다!")
        
except FileNotFoundError:
    print("❌ ESLint가 설치되지 않았습니다. npm install이 필요할 수 있습니다.")
except Exception as e:
    print(f"❌ ESLint 실행 중 오류: {e}")

# 2. 패키지 설치 확인
print(f"\n2️⃣ 필요한 도구 설치 상태 확인")
print("-" * 40)

# node_modules 확인
node_modules = project_path / "node_modules"
if node_modules.exists():
    print("✅ node_modules 디렉토리 존재")
    
    # 중요한 패키지들 확인
    important_packages = [
        "eslint",
        "@typescript-eslint/eslint-plugin", 
        "@typescript-eslint/parser",
        "typescript"
    ]
    
    for package in important_packages:
        package_path = node_modules / package
        if package_path.exists():
            print(f"  ✅ {package}")
        else:
            print(f"  ❌ {package} (설치 필요)")
else:
    print("❌ node_modules가 없습니다. npm install을 실행해주세요.")

print(f"\n3️⃣ TypeScript 컴파일 상태 확인")
print("-" * 40)

try:
    # TypeScript 컴파일러로 타입 체크
    tsc_result = subprocess.run(
        ['npx', 'tsc', '--noEmit'],
        capture_output=True,
        text=True,
        cwd=str(project_path),
        shell=True
    )
    
    if tsc_result.returncode == 0:
        print("✅ TypeScript 컴파일 성공 - 타입 에러 없음")
    else:
        lines = tsc_result.stdout.split('\n')
        error_lines = [line for line in lines if line.strip() and not line.startswith('Found')]
        print(f"⚠️  TypeScript 컴파일 에러 {len(error_lines)}개 발견")
        
        # 처음 5개 에러만 표시
        for line in error_lines[:5]:
            if line.strip():
                print(f"  - {line.strip()}")
        
        if len(error_lines) > 5:
            print(f"  ... 및 {len(error_lines) - 5}개 추가 에러")
            
except Exception as e:
    print(f"❌ TypeScript 컴파일러 실행 중 오류: {e}")

🛠️  자동 미사용 변수 탐지 및 제거 도구 실습

1️⃣ ESLint로 미사용 변수 에러 확인
----------------------------------------


Exception in thread Thread-3 (_readerthread):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\profe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 29291: illegal multibyte sequence


❌ ESLint 실행 중 오류: the JSON object must be str, bytes or bytearray, not NoneType

2️⃣ 필요한 도구 설치 상태 확인
----------------------------------------
✅ node_modules 디렉토리 존재
  ✅ eslint
  ✅ @typescript-eslint/eslint-plugin
  ✅ @typescript-eslint/parser
  ✅ typescript

3️⃣ TypeScript 컴파일 상태 확인
----------------------------------------
⚠️  TypeScript 컴파일 에러 15개 발견
  - src/bounded-contexts/donation/infrastructure/repositories/supabase-donation.repository.ts(560,9): error TS2416: Property 'findByInstituteId' in type 'SupabaseDonationRepository' is not assignable to the same property in base type 'IDonationRepository'.
  - Type '(_instituteId: string, _pagination: PaginationParams) => Promise<Result<PaginatedResult<Donation>>>' is not assignable to type '(instituteId: InstituteId, pagination?: PaginationParams | undefined) => Promise<Result<PaginatedResult<Donation>>>'.
  - Types of parameters '_instituteId' and 'instituteId' are incompatible.
  - Type 'InstituteId' is not assignable to type 'string'.

In [7]:
# 미사용 변수 자동 수정 스크립트 실습

print("🔧 미사용 변수 자동 수정 스크립트")
print("=" * 50)

def auto_fix_unused_vars(file_path: str, dry_run: bool = True):
    """
    미사용 변수 자동 수정 함수
    
    Args:
        file_path: 수정할 파일 경로
        dry_run: True면 실제 수정하지 않고 결과만 출력
    """
    
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    lines = content.splitlines()
    modified_lines = []
    changes_made = []
    
    for i, line in enumerate(lines):
        original_line = line
        
        # 1. 미사용 매개변수 처리 (_로 시작하지 않는 경우 _를 추가)
        import re
        
        # 함수 매개변수에서 미사용된 것들 찾기 (간단한 패턴)
        # 실제로는 더 정교한 AST 분석이 필요하지만, 여기서는 기본 패턴만
        
        # 예: unused_param: Type => _unused_param: Type
        pattern = r'(\w+)(\s*:\s*\w+)(?=\s*[,)])'
        
        if re.search(r'function\s+\w+\s*\(', line) or re.search(r'\(\s*\w+\s*:', line):
            # 매개변수가 포함된 라인인 경우
            if any(keyword in line for keyword in ['unused', 'config', 'params']) and not line.strip().startswith('_'):
                # 간단한 규칙: unused가 포함된 매개변수명 앞에 _ 추가
                modified_line = re.sub(r'\b(unused\w*|config|params)(\s*:)', r'_\1\2', line)
                if modified_line != original_line:
                    changes_made.append({
                        'line_num': i + 1,
                        'original': original_line.strip(),
                        'modified': modified_line.strip(),
                        'type': 'parameter_rename'
                    })
                    line = modified_line
        
        # 2. 미사용 변수 선언 제거 (const unused_var = ...)
        if re.search(r'const\s+\w*unused\w*\s*=', line) and 'console.log' not in line:
            if dry_run:
                changes_made.append({
                    'line_num': i + 1,
                    'original': original_line.strip(),
                    'modified': '// ' + original_line.strip() + ' // REMOVED: unused variable',
                    'type': 'variable_removal'
                })
                line = '// ' + line + ' // REMOVED: unused variable'
            else:
                # 실제 수정 시에는 라인을 제거하거나 주석 처리
                line = '// ' + line + ' // REMOVED: unused variable'
        
        # 3. 불필요한 console.log 제거 (임시 디버깅 코드)
        if 'console.log' in line and any(keyword in line for keyword in ['unused', 'debug', 'test']):
            if dry_run:
                changes_made.append({
                    'line_num': i + 1,
                    'original': original_line.strip(),
                    'modified': '// ' + original_line.strip() + ' // REMOVED: debug code',
                    'type': 'debug_removal'
                })
                line = '// ' + line + ' // REMOVED: debug code'
        
        modified_lines.append(line)
    
    if dry_run:
        return changes_made, '\n'.join(modified_lines)
    else:
        # 실제 파일 수정
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(modified_lines))
        return changes_made, None

# 테스트할 파일 선택
test_files = [
    "src/bounded-contexts/prediction/domain/services/incentive-alignment.service.ts",
    "src/bounded-contexts/prediction/domain/services/pmp-pmc-conversion.engine.ts"
]

for relative_path in test_files:
    file_path = project_path / relative_path
    
    if file_path.exists():
        print(f"\n📁 파일: {relative_path}")
        print("-" * 60)
        
        # Dry run으로 수정 사항 미리보기
        changes, preview = auto_fix_unused_vars(str(file_path), dry_run=True)
        
        if changes:
            print(f"🔄 {len(changes)}개의 수정 사항 발견:")
            
            for change in changes[:5]:  # 최대 5개만 표시
                print(f"\n  📍 라인 {change['line_num']} ({change['type']}):")
                print(f"    기존: {change['original']}")
                print(f"    수정: {change['modified']}")
            
            if len(changes) > 5:
                print(f"    ... 및 {len(changes) - 5}개 추가 수정사항")
        else:
            print("✅ 자동 수정할 항목이 없습니다.")
    else:
        print(f"❌ 파일을 찾을 수 없습니다: {relative_path}")

# ESLint --fix 실행 시뮬레이션
print(f"\n🛠️  ESLint --fix 실행 시뮬레이션")
print("-" * 50)

try:
    # 특정 파일에 대해 ESLint --fix 실행
    target_file = "src/bounded-contexts/prediction/domain/services/incentive-alignment.service.ts"
    
    # 실제로는 dry-run으로 실행 (--fix-dry-run 옵션)
    fix_result = subprocess.run(
        ['npx', 'eslint', target_file, '--fix-dry-run', '--format', 'json'],
        capture_output=True,
        text=True,
        cwd=str(project_path),
        shell=True
    )
    
    if fix_result.stdout:
        try:
            fix_data = json.loads(fix_result.stdout)
            print(f"🔍 ESLint --fix 분석 결과:")
            
            for file_result in fix_data:
                if file_result.get('messages'):
                    print(f"  파일: {file_result['filePath'].split('/')[-1]}")
                    fixable_count = sum(1 for msg in file_result['messages'] if msg.get('fixable'))
                    total_count = len(file_result['messages'])
                    print(f"  전체 이슈: {total_count}개")
                    print(f"  자동 수정 가능: {fixable_count}개")
                    
                    # 수정 가능한 항목들 표시
                    for msg in file_result['messages'][:3]:
                        if msg.get('fixable'):
                            print(f"    ✅ 라인 {msg['line']}: {msg['message']}")
                        else:
                            print(f"    ⚠️  라인 {msg['line']}: {msg['message']}")
        except json.JSONDecodeError:
            print("⚠️  ESLint 결과를 파싱할 수 없습니다.")
    else:
        print("✅ ESLint --fix로 수정할 항목이 없거나 이미 수정되었습니다.")
        
except Exception as e:
    print(f"❌ ESLint --fix 실행 중 오류: {e}")

🔧 미사용 변수 자동 수정 스크립트

📁 파일: src/bounded-contexts/prediction/domain/services/incentive-alignment.service.ts
------------------------------------------------------------
✅ 자동 수정할 항목이 없습니다.

📁 파일: src/bounded-contexts/prediction/domain/services/pmp-pmc-conversion.engine.ts
------------------------------------------------------------
✅ 자동 수정할 항목이 없습니다.

🛠️  ESLint --fix 실행 시뮬레이션
--------------------------------------------------


Exception in thread Thread-7 (_readerthread):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\profe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 397: illegal multibyte sequence


✅ ESLint --fix로 수정할 항목이 없거나 이미 수정되었습니다.


## 6. VSCode에서 미사용 변수 자동 수정 실습

### 6.1 VSCode Extension 활용
- **ESLint Extension**: 실시간 에러 탐지 및 Quick Fix 제공
- **TypeScript Hero**: 미사용 import 자동 정리
- **Auto Import - ES6**: import 문 자동 관리
- **TypeScript Importer**: 자동 import 추가/제거

### 6.2 VSCode Quick Fix 기능
1. **Ctrl+.** (Quick Fix): 현재 커서 위치의 문제 자동 수정
2. **Shift+Alt+.** (Auto Fix): 파일 전체 자동 수정 가능한 문제들 일괄 처리  
3. **F8**: 다음 문제로 이동
4. **Shift+F8**: 이전 문제로 이동

### 6.3 실시간 처리 설정
```json
// .vscode/settings.json
{
  "editor.codeActionsOnSave": {
    "source.fixAll.eslint": true,
    "source.organizeImports": true,
    "source.removeUnusedImports": true
  },
  "eslint.autoFixOnSave": true,
  "typescript.preferences.removeUnusedImports": true
}
```

### 6.4 VSCode 명령 팔레트 활용
- **ESLint: Fix all auto-fixable Problems**: 전체 파일 자동 수정
- **TypeScript: Organize Imports**: import 문 정리
- **TypeScript: Remove Unused Imports**: 미사용 import 제거

In [6]:
# VSCode 설정 파일 생성 및 자동화 실습

import json
import os

print("⚙️  VSCode 설정 파일 생성 및 자동화")
print("=" * 50)

# 1. .vscode 디렉토리 생성
vscode_dir = project_path / ".vscode"
vscode_dir.mkdir(exist_ok=True)

print(f"📁 VSCode 설정 디렉토리: {vscode_dir}")

# 2. settings.json 생성/수정
settings_file = vscode_dir / "settings.json"

# 미사용 변수 자동 수정을 위한 권장 설정
recommended_settings = {
    "editor.codeActionsOnSave": {
        "source.fixAll.eslint": "explicit",
        "source.organizeImports": "explicit", 
        "source.removeUnusedImports": "explicit"
    },
    "eslint.autoFixOnSave": True,
    "typescript.preferences.removeUnusedImports": True,
    "typescript.suggest.autoImports": "on",
    "typescript.updateImportsOnFileMove.enabled": "always",
    "editor.formatOnSave": True,
    "editor.defaultFormatter": "esbenp.prettier-vscode",
    "eslint.validate": [
        "javascript",
        "javascriptreact", 
        "typescript",
        "typescriptreact"
    ],
    "eslint.workingDirectories": ["./"],
    "typescript.preferences.includePackageJsonAutoImports": "on",
    "editor.quickSuggestions": {
        "other": True,
        "comments": False,
        "strings": False
    }
}

# 기존 설정이 있다면 병합
existing_settings = {}
if settings_file.exists():
    try:
        with open(settings_file, 'r', encoding='utf-8') as f:
            existing_settings = json.load(f)
        print("✅ 기존 settings.json 발견 - 설정을 병합합니다.")
    except json.JSONDecodeError:
        print("⚠️  기존 settings.json이 유효하지 않습니다. 새로 생성합니다.")

# 설정 병합
merged_settings = {**existing_settings, **recommended_settings}

# settings.json 작성
with open(settings_file, 'w', encoding='utf-8') as f:
    json.dump(merged_settings, f, indent=2, ensure_ascii=False)

print(f"✅ settings.json 생성/업데이트 완료")
print(f"   파일 위치: {settings_file}")

# 3. tasks.json 생성 (ESLint 자동 수정 태스크)
tasks_file = vscode_dir / "tasks.json"

tasks_config = {
    "version": "2.0.0",
    "tasks": [
        {
            "label": "ESLint: Fix All Files",
            "type": "shell",
            "command": "npx",
            "args": ["eslint", "src/", "--fix"],
            "group": {
                "kind": "build",
                "isDefault": False
            },
            "presentation": {
                "echo": True,
                "reveal": "always",
                "focus": False,
                "panel": "shared"
            },
            "problemMatcher": ["$eslint-stylish"]
        },
        {
            "label": "TypeScript: Check Types",
            "type": "shell", 
            "command": "npx",
            "args": ["tsc", "--noEmit"],
            "group": {
                "kind": "test",
                "isDefault": True
            },
            "presentation": {
                "echo": True,
                "reveal": "always",
                "focus": False,
                "panel": "shared"
            },
            "problemMatcher": ["$tsc"]
        },
        {
            "label": "Remove Unused Variables (Auto)",
            "type": "shell",
            "command": "npx",
            "args": [
                "eslint", 
                "src/",
                "--fix",
                "--rule",
                "@typescript-eslint/no-unused-vars:error"
            ],
            "group": "build",
            "presentation": {
                "echo": True,
                "reveal": "always",
                "focus": True,
                "panel": "shared"
            },
            "problemMatcher": ["$eslint-stylish"]
        }
    ]
}

with open(tasks_file, 'w', encoding='utf-8') as f:
    json.dump(tasks_config, f, indent=2, ensure_ascii=False)

print(f"✅ tasks.json 생성 완료")
print(f"   파일 위치: {tasks_file}")

# 4. extensions.json 생성 (권장 확장 프로그램)
extensions_file = vscode_dir / "extensions.json"

extensions_config = {
    "recommendations": [
        "dbaeumer.vscode-eslint",
        "esbenp.prettier-vscode",
        "ms-vscode.vscode-typescript-next",
        "bradlc.vscode-tailwindcss",
        "formulahendry.auto-rename-tag",
        "christian-kohler.path-intellisense",
        "ms-vscode.vscode-json"
    ],
    "unwantedRecommendations": []
}

with open(extensions_file, 'w', encoding='utf-8') as f:
    json.dump(extensions_config, f, indent=2, ensure_ascii=False)

print(f"✅ extensions.json 생성 완료")
print(f"   파일 위치: {extensions_file}")

# 5. keybindings.json 생성 (단축키 설정)
keybindings_file = vscode_dir / "keybindings.json"

keybindings_config = [
    {
        "key": "ctrl+shift+f",
        "command": "eslint.executeAutofix",
        "when": "editorTextFocus && !editorReadonly"
    },
    {
        "key": "ctrl+alt+o",
        "command": "editor.action.organizeImports",
        "when": "editorTextFocus && !editorReadonly && resourceExtname =~ /\\.(ts|tsx|js|jsx)$/"
    },
    {
        "key": "ctrl+alt+r",
        "command": "typescript.removeUnusedImports",
        "when": "editorTextFocus && !editorReadonly && resourceExtname =~ /\\.(ts|tsx)$/"
    }
]

with open(keybindings_file, 'w', encoding='utf-8') as f:
    json.dump(keybindings_config, f, indent=2, ensure_ascii=False)

print(f"✅ keybindings.json 생성 완료")

# 6. launch.json 생성 (디버깅 설정)
launch_file = vscode_dir / "launch.json"

if not launch_file.exists():
    launch_config = {
        "version": "0.2.0",
        "configurations": [
            {
                "name": "Debug TypeScript",
                "type": "node",
                "request": "launch",
                "program": "${workspaceFolder}/node_modules/.bin/ts-node",
                "args": ["${file}"],
                "cwd": "${workspaceFolder}",
                "protocol": "inspector",
                "restart": True,
                "env": {
                    "NODE_ENV": "development"
                },
                "console": "integratedTerminal",
                "internalConsoleOptions": "neverOpen"
            }
        ]
    }
    
    with open(launch_file, 'w', encoding='utf-8') as f:
        json.dump(launch_config, f, indent=2, ensure_ascii=False)
    
    print(f"✅ launch.json 생성 완료")

print(f"\n🎯 VSCode 설정 완료!")
print(f"다음 단축키를 사용하여 미사용 변수를 빠르게 수정할 수 있습니다:")
print(f"  - Ctrl+Shift+F: ESLint 자동 수정")
print(f"  - Ctrl+Alt+O: Import 정리")  
print(f"  - Ctrl+Alt+R: 미사용 Import 제거")
print(f"  - Ctrl+.: Quick Fix (현재 라인)")
print(f"  - F8: 다음 문제로 이동")

⚙️  VSCode 설정 파일 생성 및 자동화
📁 VSCode 설정 디렉토리: c:\G\posmul\.vscode
✅ settings.json 생성/업데이트 완료
   파일 위치: c:\G\posmul\.vscode\settings.json
✅ tasks.json 생성 완료
   파일 위치: c:\G\posmul\.vscode\tasks.json
✅ extensions.json 생성 완료
   파일 위치: c:\G\posmul\.vscode\extensions.json
✅ keybindings.json 생성 완료
✅ launch.json 생성 완료

🎯 VSCode 설정 완료!
다음 단축키를 사용하여 미사용 변수를 빠르게 수정할 수 있습니다:
  - Ctrl+Shift+F: ESLint 자동 수정
  - Ctrl+Alt+O: Import 정리
  - Ctrl+Alt+R: 미사용 Import 제거
  - Ctrl+.: Quick Fix (현재 라인)
  - F8: 다음 문제로 이동


## 7. CI/CD에서 ESLint 검사 자동화

### 7.1 GitHub Actions 설정
프로젝트에 GitHub Actions를 설정하여 Pull Request 시 자동으로 미사용 변수를 검사합니다.

### 7.2 ESLint 검사 단계
1. **lint**: 코드 품질 검사
2. **type-check**: TypeScript 타입 검사  
3. **build**: 빌드 가능성 확인
4. **test**: 단위 테스트 실행

### 7.3 자동 수정 vs 검사 전용
- **검사 전용**: PR에서 문제만 보고하고 개발자가 수동 수정
- **자동 수정**: CI에서 자동으로 수정하고 커밋 생성 (위험할 수 있음)
- **권장 방식**: 검사 + 실패 시 상세 보고서 제공

### 7.4 Pre-commit Hook 설정
```bash
# husky와 lint-staged 사용
npm install --save-dev husky lint-staged
npx husky install
npx husky add .husky/pre-commit "npx lint-staged"
```

In [ ]:
# CI/CD 설정 파일 생성 및 자동화

import os
import yaml

print("🚀 CI/CD ESLint 자동화 설정")
print("=" * 50)

# 1. GitHub Actions 워크플로우 생성
github_dir = project_path / ".github"
workflows_dir = github_dir / "workflows"
workflows_dir.mkdir(parents=True, exist_ok=True)

# CI 워크플로우 설정
ci_workflow = {
    'name': 'CI - Code Quality Check',
    'on': {
        'push': {
            'branches': ['main', 'develop']
        },
        'pull_request': {
            'branches': ['main', 'develop']
        }
    },
    'jobs': {
        'lint-and-typecheck': {
            'runs-on': 'ubuntu-latest',
            'strategy': {
                'matrix': {
                    'node-version': ['18.x', '20.x']
                }
            },
            'steps': [
                {
                    'name': 'Checkout code',
                    'uses': 'actions/checkout@v4'
                },
                {
                    'name': 'Setup Node.js',
                    'uses': 'actions/setup-node@v4',
                    'with': {
                        'node-version': '${{ matrix.node-version }}',
                        'cache': 'npm'
                    }
                },
                {
                    'name': 'Install dependencies',
                    'run': 'npm ci'
                },
                {
                    'name': 'Run ESLint',
                    'run': 'npm run lint'
                },
                {
                    'name': 'Check TypeScript types',
                    'run': 'npm run type-check'
                },
                {
                    'name': 'Run tests',
                    'run': 'npm test'
                },
                {
                    'name': 'Build project',
                    'run': 'npm run build'
                },
                {
                    'name': 'Upload ESLint results',
                    'uses': 'actions/upload-artifact@v3',
                    'if': 'failure()',
                    'with': {
                        'name': 'eslint-results',
                        'path': 'eslint-report.json'
                    }
                }
            ]
        }
    }
}

ci_file = workflows_dir / "ci.yml"
with open(ci_file, 'w', encoding='utf-8') as f:
    yaml.dump(ci_workflow, f, default_flow_style=False, allow_unicode=True, sort_keys=False)

print(f"✅ GitHub Actions CI 워크플로우 생성: {ci_file}")

# 2. 코드 품질 검사 워크플로우 (더 상세한 버전)
quality_workflow = {
    'name': 'Code Quality Analysis',
    'on': {
        'pull_request': {
            'branches': ['main']
        }
    },
    'jobs': {
        'code-quality': {
            'runs-on': 'ubuntu-latest',
            'steps': [
                {
                    'name': 'Checkout',
                    'uses': 'actions/checkout@v4'
                },
                {
                    'name': 'Setup Node.js',
                    'uses': 'actions/setup-node@v4',
                    'with': {
                        'node-version': '20.x',
                        'cache': 'npm'
                    }
                },
                {
                    'name': 'Install dependencies',
                    'run': 'npm ci'
                },
                {
                    'name': 'Run ESLint with detailed report',
                    'run': '''
                      npx eslint src/ --format json --output-file eslint-report.json || true
                      npx eslint src/ --format stylish
                    '''
                },
                {
                    'name': 'Generate unused variables report',
                    'run': '''
                      echo "## 🔍 미사용 변수 검사 결과" >> $GITHUB_STEP_SUMMARY
                      echo "\\`\\`\\`" >> $GITHUB_STEP_SUMMARY
                      npx eslint src/ --format unix | grep "no-unused-vars" | head -20 >> $GITHUB_STEP_SUMMARY || echo "미사용 변수 없음" >> $GITHUB_STEP_SUMMARY
                      echo "\\`\\`\\`" >> $GITHUB_STEP_SUMMARY
                    '''
                },
                {
                    'name': 'TypeScript strict check',
                    'run': 'npx tsc --noEmit --strict'
                },
                {
                    'name': 'Comment PR with results',
                    'uses': 'actions/github-script@v7',
                    'if': 'failure()',
                    'with': {
                        'script': '''
                          const fs = require('fs');
                          let eslintReport = '';
                          try {
                            const report = JSON.parse(fs.readFileSync('eslint-report.json', 'utf8'));
                            const unusedVarErrors = report.flatMap(file => 
                              file.messages.filter(msg => msg.ruleId === '@typescript-eslint/no-unused-vars')
                            );
                            
                            if (unusedVarErrors.length > 0) {
                              eslintReport = `## 🚨 미사용 변수 발견 (${unusedVarErrors.length}개)\\n\\n`;
                              eslintReport += unusedVarErrors.slice(0, 10).map(error => 
                                `- \${error.message} (라인 \${error.line})`
                              ).join('\\n');
                              
                              if (unusedVarErrors.length > 10) {
                                eslintReport += `\\n\\n... 및 \${unusedVarErrors.length - 10}개 추가`;
                              }
                            }
                          } catch (e) {
                            eslintReport = '리포트 생성 중 오류가 발생했습니다.';
                          }
                          
                          github.rest.issues.createComment({
                            issue_number: context.issue.number,
                            owner: context.repo.owner,
                            repo: context.repo.repo,
                            body: eslintReport || '✅ 미사용 변수가 없습니다!'
                          });
                        '''
                    }
                }
            ]
        }
    }
}

quality_file = workflows_dir / "code-quality.yml"
with open(quality_file, 'w', encoding='utf-8') as f:
    yaml.dump(quality_workflow, f, default_flow_style=False, allow_unicode=True, sort_keys=False)

print(f"✅ 코드 품질 분석 워크플로우 생성: {quality_file}")

# 3. package.json에 필요한 스크립트 추가
package_json_file = project_path / "package.json"

if package_json_file.exists():
    with open(package_json_file, 'r', encoding='utf-8') as f:
        package_data = json.load(f)
    
    # 필요한 스크립트들 추가/업데이트
    scripts = package_data.setdefault('scripts', {})
    
    recommended_scripts = {
        'lint': 'eslint src/ --ext .ts,.tsx',
        'lint:fix': 'eslint src/ --ext .ts,.tsx --fix',
        'lint:report': 'eslint src/ --ext .ts,.tsx --format json --output-file eslint-report.json',
        'type-check': 'tsc --noEmit',
        'type-check:strict': 'tsc --noEmit --strict',
        'quality-check': 'npm run lint && npm run type-check',
        'quality-fix': 'npm run lint:fix && npm run type-check'
    }
    
    # 기존 스크립트와 병합
    scripts.update(recommended_scripts)
    
    # package.json 업데이트
    with open(package_json_file, 'w', encoding='utf-8') as f:
        json.dump(package_data, f, indent=2, ensure_ascii=False)
    
    print("✅ package.json 스크립트 업데이트 완료")
    print("추가된 스크립트:")
    for script, command in recommended_scripts.items():
        print(f"  - npm run {script}: {command}")

# 4. Pre-commit 설정 파일 생성
husky_dir = project_path / ".husky"
husky_dir.mkdir(exist_ok=True)

# pre-commit 훅 생성
precommit_file = husky_dir / "pre-commit"
precommit_content = '''#!/usr/bin/env sh
. "$(dirname -- "$0")/_/husky.sh"

# ESLint 검사 실행
echo "🔍 ESLint 검사 중..."
npm run lint

# TypeScript 타입 검사
echo "🔧 TypeScript 타입 검사 중..."
npm run type-check

echo "✅ Pre-commit 검사 완료!"
'''

with open(precommit_file, 'w', encoding='utf-8') as f:
    f.write(precommit_content)

# 실행 권한 부여 (Windows에서는 불필요하지만 크로스 플랫폼 호환성을 위해)
if os.name != 'nt':  # Windows가 아닌 경우
    os.chmod(precommit_file, 0o755)

print(f"✅ Pre-commit 훅 생성: {precommit_file}")

# 5. .gitignore에 ESLint 리포트 파일 추가
gitignore_file = project_path / ".gitignore"

if gitignore_file.exists():
    with open(gitignore_file, 'r', encoding='utf-8') as f:
        gitignore_content = f.read()
    
    # ESLint 리포트 관련 항목들
    eslint_ignores = [
        "# ESLint reports",
        "eslint-report.json",
        "eslint-report.html",
        ".eslintcache"
    ]
    
    # 중복 확인 후 추가
    for ignore_item in eslint_ignores:
        if ignore_item not in gitignore_content:
            gitignore_content += f"\n{ignore_item}"
    
    with open(gitignore_file, 'w', encoding='utf-8') as f:
        f.write(gitignore_content)
    
    print("✅ .gitignore 업데이트 완료")

print(f"\n🎯 CI/CD 자동화 설정 완료!")
print(f"설정된 기능들:")
print(f"  ✅ GitHub Actions CI 워크플로우")
print(f"  ✅ 코드 품질 분석 자동화")
print(f"  ✅ PR 코멘트 자동 리포팅")
print(f"  ✅ Pre-commit 훅")
print(f"  ✅ package.json 스크립트 확장")
print(f"  ✅ .gitignore 업데이트")

print(f"\n🚀 다음 명령어로 로컬에서 테스트할 수 있습니다:")
print(f"  npm run quality-check  # 전체 품질 검사")
print(f"  npm run quality-fix    # 자동 수정 + 검사")
print(f"  npm run lint:report    # 상세 리포트 생성")

## 8. 실습 결과 종합 및 모범 사례

### 8.1 학습한 미사용 변수 해결 방법들

| 방법 | 적용 시점 | 자동화 수준 | 권장 사용 |
|------|-----------|-------------|-----------|
| **ESLint --fix** | 개발 중, CI/CD | ⭐⭐⭐ | 일반적인 미사용 변수 |
| **VSCode Quick Fix** | 실시간 개발 | ⭐⭐⭐ | 개발자 개별 수정 |
| **Pre-commit Hook** | 커밋 전 | ⭐⭐⭐ | 팀 프로젝트 필수 |
| **GitHub Actions** | PR/Push 시 | ⭐⭐⭐ | 품질 게이트 |
| **ts-prune** | 주기적 분석 | ⭐⭐ | 대규모 리팩토링 |

### 8.2 권장 개발 워크플로우

```mermaid
graph TD
    A[코드 작성] --> B[실시간 ESLint 검사]
    B --> C{에러 있음?}
    C -->|Yes| D[VSCode Quick Fix]
    C -->|No| E[커밋 시도]
    D --> E
    E --> F[Pre-commit Hook 실행]
    F --> G{검사 통과?}
    G -->|No| H[자동 수정 또는 수동 해결]
    G -->|Yes| I[커밋 완료]
    H --> F
    I --> J[Push to Remote]
    J --> K[GitHub Actions CI]
    K --> L[코드 리뷰]
```

### 8.3 프로젝트별 맞춤 설정 가이드

#### 🚀 신규 프로젝트
- 엄격한 ESLint 규칙 적용
- Pre-commit Hook 필수 설정
- CI/CD에서 빌드 실패 시 배포 중단

#### 🔧 기존 프로젝트
- 점진적 규칙 강화 (`warn` → `error`)
- 레거시 코드 단계별 개선
- 팀 합의 하에 자동 수정 도입

#### 👥 팀 프로젝트
- 공통 ESLint 설정 공유
- 코드 리뷰에서 품질 검사 결과 활용
- 정기적인 코드 품질 리포트 생성

In [8]:
# 🎯 실습 완료 - 최종 결과 확인 및 다음 단계

print("🏆 TypeScript 미사용 변수 해결 실습 완료!")
print("=" * 60)

# 1. 실습에서 생성한 파일들 확인
created_files = []

# VSCode 설정 파일들
vscode_files = [
    ".vscode/settings.json",
    ".vscode/tasks.json", 
    ".vscode/extensions.json",
    ".vscode/keybindings.json",
    ".vscode/launch.json"
]

# CI/CD 설정 파일들  
cicd_files = [
    ".github/workflows/ci.yml",
    ".github/workflows/code-quality.yml"
]

# 기타 설정 파일들
other_files = [
    ".husky/pre-commit",
    "eslint-report.json"
]

all_files = vscode_files + cicd_files + other_files

print("📁 실습에서 생성/수정한 파일들:")
print("-" * 40)

for file_path in all_files:
    full_path = project_path / file_path
    if full_path.exists():
        print(f"  ✅ {file_path}")
        created_files.append(file_path)
    else:
        print(f"  ❌ {file_path} (생성되지 않음)")

print(f"\n✅ 총 {len(created_files)}개 파일 생성/수정 완료")

# 2. 프로젝트 상태 최종 확인
print(f"\n🔍 프로젝트 최종 상태 확인")
print("-" * 40)

try:
    # ESLint 최종 검사
    final_lint_result = subprocess.run(
        ['npx', 'eslint', 'src/', '--format', 'json'],
        capture_output=True,
        text=True,
        cwd=str(project_path),
        shell=True
    )
    
    if final_lint_result.stdout:
        try:
            final_data = json.loads(final_lint_result.stdout)
            final_unused_vars = []
            
            for file_result in final_data:
                for message in file_result['messages']:
                    if 'no-unused-vars' in message.get('ruleId', ''):
                        final_unused_vars.append(message)
            
            if final_unused_vars:
                print(f"⚠️  여전히 {len(final_unused_vars)}개의 미사용 변수 에러가 남아있습니다.")
                print("   다음 명령어로 자동 수정을 시도해보세요:")
                print("   npm run lint:fix")
            else:
                print("🎉 미사용 변수 에러가 모두 해결되었습니다!")
                
        except json.JSONDecodeError:
            print("⚠️  ESLint 결과 파싱 실패")
    else:
        print("✅ ESLint 검사 통과!")
        
except Exception as e:
    print(f"⚠️  최종 검사 중 오류: {e}")

# 3. 사용 가능한 명령어 요약
print(f"\n🚀 이제 사용할 수 있는 명령어들:")
print("-" * 40)

commands = [
    ("npm run lint", "ESLint 검사 실행"),
    ("npm run lint:fix", "ESLint 자동 수정"),
    ("npm run type-check", "TypeScript 타입 검사"),
    ("npm run quality-check", "전체 품질 검사"),
    ("npm run quality-fix", "자동 수정 + 검사")
]

for cmd, desc in commands:
    print(f"  {cmd:<25} # {desc}")

# 4. VSCode 단축키 안내
print(f"\n⌨️  VSCode에서 사용할 수 있는 단축키:")
print("-" * 40)

shortcuts = [
    ("Ctrl+.", "현재 위치 Quick Fix"),
    ("Ctrl+Shift+F", "ESLint 자동 수정"),
    ("Ctrl+Alt+O", "Import 정리"),
    ("Ctrl+Alt+R", "미사용 Import 제거"),
    ("F8", "다음 문제로 이동"),
    ("Shift+F8", "이전 문제로 이동")
]

for shortcut, desc in shortcuts:
    print(f"  {shortcut:<15} # {desc}")

# 5. 다음 단계 안내
print(f"\n🎯 추천 다음 단계:")
print("-" * 40)
print("1. VSCode에서 프로젝트를 다시 열어 설정이 적용되었는지 확인")
print("2. 'npm run quality-check' 명령어로 전체 프로젝트 상태 확인")
print("3. Git 커밋을 시도하여 pre-commit hook이 작동하는지 테스트")
print("4. GitHub에 푸시하여 CI/CD 워크플로우 작동 확인")
print("5. 팀원들과 공유하여 일관된 코드 품질 관리 체계 구축")

print(f"\n📚 추가 학습 자료:")
print("-" * 40)
print("- ESLint 공식 문서: https://eslint.org/docs/rules/")
print("- TypeScript ESLint: https://typescript-eslint.io/")
print("- VSCode ESLint Extension: https://marketplace.visualstudio.com/items?itemName=dbaeumer.vscode-eslint")
print("- GitHub Actions: https://docs.github.com/en/actions")

print(f"\n🎉 실습이 성공적으로 완료되었습니다!")
print("이제 프로젝트에서 미사용 변수 문제를 체계적으로 관리할 수 있습니다.")

🏆 TypeScript 미사용 변수 해결 실습 완료!
📁 실습에서 생성/수정한 파일들:
----------------------------------------
  ✅ .vscode/settings.json
  ✅ .vscode/tasks.json
  ✅ .vscode/extensions.json
  ✅ .vscode/keybindings.json
  ✅ .vscode/launch.json
  ❌ .github/workflows/ci.yml (생성되지 않음)
  ❌ .github/workflows/code-quality.yml (생성되지 않음)
  ❌ .husky/pre-commit (생성되지 않음)
  ❌ eslint-report.json (생성되지 않음)

✅ 총 5개 파일 생성/수정 완료

🔍 프로젝트 최종 상태 확인
----------------------------------------


Exception in thread Thread-9 (_readerthread):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\profe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 29291: illegal multibyte sequence


✅ ESLint 검사 통과!

🚀 이제 사용할 수 있는 명령어들:
----------------------------------------
  npm run lint              # ESLint 검사 실행
  npm run lint:fix          # ESLint 자동 수정
  npm run type-check        # TypeScript 타입 검사
  npm run quality-check     # 전체 품질 검사
  npm run quality-fix       # 자동 수정 + 검사

⌨️  VSCode에서 사용할 수 있는 단축키:
----------------------------------------
  Ctrl+.          # 현재 위치 Quick Fix
  Ctrl+Shift+F    # ESLint 자동 수정
  Ctrl+Alt+O      # Import 정리
  Ctrl+Alt+R      # 미사용 Import 제거
  F8              # 다음 문제로 이동
  Shift+F8        # 이전 문제로 이동

🎯 추천 다음 단계:
----------------------------------------
1. VSCode에서 프로젝트를 다시 열어 설정이 적용되었는지 확인
2. 'npm run quality-check' 명령어로 전체 프로젝트 상태 확인
3. Git 커밋을 시도하여 pre-commit hook이 작동하는지 테스트
4. GitHub에 푸시하여 CI/CD 워크플로우 작동 확인
5. 팀원들과 공유하여 일관된 코드 품질 관리 체계 구축

📚 추가 학습 자료:
----------------------------------------
- ESLint 공식 문서: https://eslint.org/docs/rules/
- TypeScript ESLint: https://typescript-eslint.io/
- VSCode ESLint Extension: https://marketplace.v